---
title: "Funciones. Multiple dispatch"
subtitle: "Ejercicios"
---

## Ejercicio 1. Sucesión de Fibonacci mediante recursión

Calcule el elemento $n$-ésimo de la sucesión de Fibonacci usando recursión


## Ejercicio 2. Suma cuadrados

Escriba una función llamada `suma_cuadrados` que acepte el número natural $n$ y devuelva la suma $1^2 + 2^2 + \cdots + (n-1)^2 + n^2$

In [ ]:
using Test
@test suma_cuadrados(1) == 1
@test suma_cuadrados(2) == 9
@test suma_cuadrados(100) == 338350

## Ejercicio 3. FizzBuzz

Escribe una función llamada `fizz_buzz` que acepte un número y
- Si el número es divisible por 3 pero no 5, escriba "Fizz" (sin comillas)
- Si el número es divisible por 5 pero no 3, escriba "Buzz" (sin comillas)
- Si el número es divisible por 3 y 5, escriba "FizzBuzz" (sin comillas)
- En otro caso, escriba el propio número


In [ ]:
@test fizz_buzz(1) == 1
@test fizz_buzz(3) == "Fizz"
@test fizz_buzz(5) == "Buzz"
@test fizz_buzz(15) == "FizzBuzz"
@test fizz_buzz(16) == 16

## Ejercicio 4. El algoritmo de Euclides para polinomios, y múltiple dispatch

### El algoritmo de Euclides para números enteros

Recordamos el famoso algoritmo para calcular el mayor común divisor (*gcd* en inglés). Utiliza el resto de la división entre naturales, denotado `%`

In [ ]:
function my_gcd(a, b)
    if iszero(b)
        return a
    else
        return gcd(b, a % b)
    end
end;

Es fácil comprobar que funciona

In [ ]:
using Test
@testset "my_gcd funciona" begin
    @test my_gcd(8, 4) == 4
    @test my_gcd(21, 14) == 7
end;

La gran ventaja de `julia` es que cualquier `struct` que implemente `iszero` y `%` (también llamado `rem`) puede pasar a través de `my_gcd`. 

Disponemos de un paquete ya implementado para polinomios, que se llama [`Polynomials.jl`](https://juliamath.github.io/Polynomials.jl/stable/). 
Tiene además una representación bastante agradable de los polinomios

In [ ]:
using Polynomials
P = Polynomial([0, 1])

Además, implementa `%`

In [ ]:
P, R = Polynomial([0, 1, 1]), Polynomial([1, 0])
@test (P * P * P + R) % P == R

Veamos que efectivamente funciona esta implementación

In [ ]:
P = Polynomial([0, 1, 1])
@test my_gcd(P, P * P) == P

### Nuestra propia estructura de polinomios

Para ver que esto realmente funciona, vamos a crear nuestra propia estructura de polinomios

In [ ]:
struct Poly
    coeff::Vector{Float64} # [a₀, a₁,... , aₙ]
end

Vamos a extender la funciones `iszero` y `%`. 
La evitar problemas de comparación en `float` aprovechamos la función `iszero` para Arrays

In [ ]:
import Base: iszero, isequal

function iszero(P::Poly)
    return iszero(P.coeffs)
end

Para la división necesitaremos otras operaciones antes. Por ejemplos el grado de un polinomio

In [ ]:
function degree(P::Poly)
    if iszero(P)
        return -1
    else
        k = length(P.coeff)
        while k ≥ 1 && iszero(P.coeff[k])
            k = k - 1
        end
        return k - 1
    end
end

y la suma

In [ ]:
import Base: +
function +(a::Poly, b::Poly)
    if degree(a) < 0
        return b
    elseif degree(b) < 0
        return a
    else
        da = degree(a)
        db = degree(b)
        return Poly(
            [a.coeff[1:da+1]; zeros(max(db - da, 0))]
            +
            [b.coeff[1:db+1]; zeros(max(da - db, 0))]
        )
    end
end

Recordamos que puede implementarse la división entre polinomios mediante 

In [ ]:
function %(a::Poly, b::Poly)
    if iszero(a)
        error("¡No dividas por 0!")
    end

    # En cada paso a = b * q + r
    r = a
    while degree(r) ≥ degree(b)
        s = Poly([zeros(degree(r) - degree(b)); lead(r) / lead(b)])
        r = r - s * b
    end
    return r
end

**Ejercicio.** Extiender `-` y `*` a `Poly` para que `%` funcione, y compruebar que, entonces,`my_gcd` funciona.

In [ ]:
P = Poly([0, 1, 1])
@test my_gcd(P, P * P) == P